In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
import boto3

s3 = boto3.client('s3')

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
data_train = pd.read_csv('FEC_dataset/faceexp-comparison-data-train-public.csv',sep='\n',header=None, nrows=100)
data_test = pd.read_csv('FEC_dataset/faceexp-comparison-data-test-public.csv',sep='\n',header=None, nrows=100)
# data_test = pd.read_excel('FEC_dataset/corrected_faceexp-comparison-data-train-public.ods', engine='odf', nrows=100 )
# data_train = pd.read_excel('FEC_dataset/corrected_faceexp-comparison-data-train-public.ods', engine='odf', nrows=100)

In [4]:
data_train.columns = ['Test']
data_test.columns = ['Test']

In [5]:
data_train = data_train.Test.apply(lambda x: pd.Series(str(x).split(',')))
data_test = data_test.Test.apply(lambda x: pd.Series(str(x).split(',')))

In [7]:
data_train.columns = ['URL of image1', 'Image1 Top-left Column', 'Image1 Bottom-right Column', 'Image1 Top-Left Row', 'Image1 Bottom-right Row', 'URL of image2', 'Image2 Top-left Column', 'Image2 Bottom-right Column', 'Image2 Top-Left Row', 'Image2 Bottom-right Row','URL of image3', 'Image3 Top-left Column', 'Image3 Bottom-right Column', 'Image3 Top-Left Row', 'Image3 Bottom-right Row','Triplet Type', 'Annotater1 ID', 'Annotation1','Annotater2 ID', 'Annotation2', 'Annotater3 ID', 'Annotation3', 'Annotater4 ID', 'Annotation4', 'Annotater5 ID', 'Annotation5', 'Annotater6 ID', 'Annotation6', 'Annotater7 ID', 'Annotation7']
data_test.columns = ['URL of image1', 'Image1 Top-left Column', 'Image1 Bottom-right Column', 'Image1 Top-Left Row', 'Image1 Bottom-right Row', 'URL of image2', 'Image2 Top-left Column', 'Image2 Bottom-right Column', 'Image2 Top-Left Row', 'Image2 Bottom-right Row','URL of image3', 'Image3 Top-left Column', 'Image3 Bottom-right Column', 'Image3 Top-Left Row', 'Image3 Bottom-right Row','Triplet Type', 'Annotater1 ID', 'Annotation1','Annotater2 ID', 'Annotation2', 'Annotater3 ID', 'Annotation3', 'Annotater4 ID', 'Annotation4', 'Annotater5 ID', 'Annotation5', 'Annotater6 ID', 'Annotation6']

In [8]:
data_test.head()

,URL of image1,Image1 Top-left Column,Image1 Bottom-right Column,Image1 Top-Left Row,Image1 Bottom-right Row,URL of image2,Image2 Top-left Column,Image2 Bottom-right Column,Image2 Top-Left Row,Image2 Bottom-right Row,...,Annotater2 ID,Annotation2,Annotater3 ID,Annotation3,Annotater4 ID,Annotation4,Annotater5 ID,Annotation5,Annotater6 ID,Annotation6
0,http://farm4.staticflickr.com/3679/12137399835_d9075d3194_b.jpg,0.453125,0.655273,0.079019,0.407357,"""http://farm4.staticflickr.com/3372/5791826985_e285030df5_b.jpg""",0.523734,0.675633,0.155660,0.419811,...,300701,3,321902,3,322102,3,4294978797,3,4294989896,3
1,http://farm4.staticflickr.com/3679/12137399835_d9075d3194_b.jpg,0.453125,0.655273,0.079019,0.407357,"""http://farm6.staticflickr.com/5532/11255277973_d67d4466a5_b.jpg""",0.302734,0.568359,0.060150,0.535338,...,300701,1,300901,3,321501,3,321902,2,322102,3
2,http://farm4.staticflickr.com/3679/12137399835_d9075d3194_b.jpg,0.453125,0.655273,0.079019,0.407357,"""http://farm1.staticflickr.com/1/186137601_d1b8896958_b.jpg""",0.311558,0.444724,0.178711,0.298828,...,291701,3,300701,3,321501,3,321601,3,321902,3
3,http://farm4.staticflickr.com/3679/12137399835_d9075d3194_b.jpg,0.453125,0.655273,0.079019,0.407357,"""http://farm8.staticflickr.com/7345/11272369166_ec7f4b51ef_b.jpg""",0.185547,0.260742,0.146245,0.263505,...,291701,2,321902,3,322102,2,4294989896,2,48316,3
4,http://farm4.staticflickr.com/3679/12137399835_d9075d3194_b.jpg,0.453125,0.655273,0.079019,0.407357,"""http://farm4.staticflickr.com/3760/13641363095_c0294ab0b8_b.jpg""",0.408854,0.518229,0.277344,0.372070,...,291701,2,321601,1,322102,2,4294989896,2,48316,2


In [9]:
pd.unique(data_train['URL of image1'])

array(['http://farm5.staticflickr.com/4108/5185055338_1dec873bf3_b.jpg',
       'http://farm8.staticflickr.com/7458/11756029544_0c4ab1713c_b.jpg',
       'http://farm6.staticflickr.com/5451/9670218976_ee459547ff_b.jpg',
       'http://farm2.staticflickr.com/1194/800391570_bece0a761c_b.jpg',
       'http://farm6.staticflickr.com/5033/14414157667_98383c7f1c_b.jpg',
       'http://farm3.staticflickr.com/2938/14716663066_5a80897c22_b.jpg',
       'http://farm4.staticflickr.com/3732/11190266664_e1cce12d3f_b.jpg',
       'http://farm2.staticflickr.com/1421/4721793735_d23bc3e9a3_b.jpg',
       'http://farm8.staticflickr.com/7354/10667751636_a2e87c193e_b.jpg',
       'http://farm5.staticflickr.com/4037/4479925876_d60c925bd5_b.jpg',
       'http://farm7.staticflickr.com/6078/6049791236_1c994ccd2e_b.jpg',
       'http://farm3.staticflickr.com/2057/2116299821_a97d441552_b.jpg',
       'http://farm4.staticflickr.com/3089/2633070690_e7b1e66c2f_z.jpg'],
      dtype=object)

In [11]:
df_train = data_train[['URL of image1', 'Image1 Top-left Column', 'Image1 Bottom-right Column', 'Image1 Top-Left Row', 'Image1 Bottom-right Row']])
df_train
df_train.head()

ValueError: could not broadcast input array from shape (100,5) into shape (100)

In [10]:
s3.list_buckets()['Buckets']

[{'Name': 'aws.bucket123456',
  'CreationDate': datetime.datetime(2020, 10, 19, 22, 55, 16, tzinfo=tzutc())},
 {'Name': 'sagemaker-studio-fqqnc829sdw',
  'CreationDate': datetime.datetime(2020, 12, 22, 19, 28, 9, tzinfo=tzutc())},
 {'Name': 'wmc-beedle',
  'CreationDate': datetime.datetime(2020, 10, 19, 21, 20, 4, tzinfo=tzutc())}]

In [14]:
for obj in s3.list_objects_v2(Bucket='sagemaker-studio-fqqnc829sdw')['Contents']:
    print(obj['Key'])

FEC_dataset/FEC_dataset.pdf
FEC_dataset/corrected-faceexp-comparison-data-test-public.ods
FEC_dataset/corrected_faceexp-comparison-data-train-public.ods
FEC_dataset/faceexp-comparison-data-test-public.csv
FEC_dataset/faceexp-comparison-data-train-public.csv


In [15]:
response = s3.get_object(Bucket='sagemaker-studio-fqqnc829sdw', Key='FEC_dataset/faceexp-comparison-data-train-public.csv')

In [16]:
response

{'ResponseMetadata': {'RequestId': '05860AA18B69BFAF',
  'HostId': 'tfE7ZBvYEkDGjCgciiit+jW4Ve8K+3cEHSqPi0BSDKA4BFCRbBDj71RdaxO1CJIR2g7W1MohbRI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'tfE7ZBvYEkDGjCgciiit+jW4Ve8K+3cEHSqPi0BSDKA4BFCRbBDj71RdaxO1CJIR2g7W1MohbRI=',
   'x-amz-request-id': '05860AA18B69BFAF',
   'date': 'Mon, 04 Jan 2021 21:54:25 GMT',
   'last-modified': 'Mon, 04 Jan 2021 19:10:36 GMT',
   'etag': '"4a1ab158af721e9af49e15db9b430302-10"',
   'x-amz-version-id': 'Mk4ECrPeCD1LLNBgrfHttz8X5Rysmfmc',
   'accept-ranges': 'bytes',
   'content-type': 'text/csv',
   'content-length': '170188378',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2021, 1, 4, 19, 10, 36, tzinfo=tzutc()),
 'ContentLength': 170188378,
 'ETag': '"4a1ab158af721e9af49e15db9b430302-10"',
 'VersionId': 'Mk4ECrPeCD1LLNBgrfHttz8X5Rysmfmc',
 'ContentType': 'text/csv',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0